# Consignador NN - Regresion Logistica

## Contenido
1. [Importar librerias](#Importar-librerias)
2. [Cargar datos](#Cargar-datos)
3. [Preparar datos](#Preparar-datos)
4. [Modelo](#Modelo)
5. [Entrenamiento](#Entrenamiento) (#TODO: agregar grafico de perdida y guardar mejor modelo en cada epoca)
6. [Validacion](#Validacion)
7. [Prediccion](#Prediccion)
8. [Guardar modelo](#Guardar-modelo)


## Importar librerias

In [1]:
# Importar librerias
import utilities as ut
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
from keras.models import load_model, Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.metrics import AUC, Precision, Recall



## Cargar datos

In [2]:
data = pd.read_pickle(ut.INCIDENTES_CAMARAS_FILENAME)
# convertir el id_camara a int
data['id_camara'] = data['id_camara'].astype(int)
data

,folio,incidente_c4,fecha_creacion,sector_inicio,delegacion_inicio,colonia,latitud,longitud,id_camara
0,C2C/20220112/00030,Administrativas-Drogados,2022-01-12 08:49:38,ALAMEDA,CUAUHTEMOC,CENTRO (AREA 5),19.435283,-99.147152,1
1,C5/20220118/00090,Robo-Transeúnte,2022-01-18 00:49:47,ALAMEDA,CUAUHTEMOC,CENTRO (AREA 5),19.435149,-99.147325,1
2,C2C/20230108/00178,Disturbio-Riña,2023-01-08 20:47:51,ALAMEDA,CUAUHTEMOC,CENTRO (AREA 5),19.435283,-99.147152,1
3,C5/20230122/01336,Administrativas-Ebrios,2023-01-22 05:10:07,ALAMEDA,CUAUHTEMOC,CENTRO (AREA 5),19.435149,-99.147325,1
4,C2C/20220202/00008,Administrativas-Ebrios,2022-02-02 01:19:04,ALAMEDA,CUAUHTEMOC,CENTRO (AREA 5),19.435283,-99.147152,1
...,...,...,...,...,...,...,...,...,...
31972,C2N/20210727/00036,Robo-Transeúnte,2021-07-27 10:06:22,CONGRESO,VENUSTIANO CARRANZA,AMPLIACION PENITENCIARIA,19.435700,-99.116500,17290
31973,C5/20220814/00067,Disturbio-Escándalo,2022-08-14 00:07:55,CONGRESO,VENUSTIANO CARRANZA,AMPLIACION PENITENCIARIA,19.435692,-99.116920,17290
31974,C2N/20210906/00030,Robo-Establecimiento con Violencia,2021-09-06 11:43:53,CONGRESO,VENUSTIANO CARRANZA,AMPLIACION PENITENCIARIA,19.435486,-99.115698,17290
31975,C2N/20220902/00049,Robo-Transeúnte,2022-09-02 10:11:05,CONGRESO,VENUSTIANO CARRANZA,AMPLIACION PENITENCIARIA,19.435700,-99.116500,17290


#### Cargar datos de puntos de interes

In [3]:
DIR_PATH = '\\\\C4wadpninv004\\ANALISIS II-DGGE\\02. SME\\GUSTAVO\\'
# DIR_PATH = '..\\data\\'
FILE_PATH = DIR_PATH + 'B200m_CONSIGNADAS.csv'
FILE_PATH = DIR_PATH + 'B200m_CONSIGNADAS-v3.csv'
data_interest_points = pd.read_csv(FILE_PATH, sep=',', encoding='latin-1')
# Renombrar la columna 'Etiquetas' a 'id_camara'
data_interest_points.rename(columns={'id': 'id_camara'}, inplace=True)
# Ordenar los datos por la columna 'id_camara'
data_interest_points.sort_values(by=['id_camara'], inplace=True)
# Resetear los índices
data_interest_points.reset_index(drop=True, inplace=True)

data_interest_points

,id_camara,latitud,longitud,sector,tipo,C01-BANCOS,C05-CENTRALES CAMIONERAS,C06-CENTROS COMERCIALES,C09-CUARTELES DE LA POLICÍA AUXILIAR,C10-CUARTELES PBI,...,C62-OXXO,C67-CENTROS PILARES,C72-ACCESOS METRO,C73-ATRACTIVOS TURISTICOS,C75-FONOTECAS,C76-FOTOTECA,C77-GALERIAS,C78-ZONAS ARQUEOLOGICAS,C81-ESTACIONES DE CABLEBUS,C82-ALCALDIAS
0,1,19.435283,-99.147152,ALAMEDA,9m,25,0,1,0,0,...,4,0,1,3,0,0,2,0,0,0
1,2,19.435098,-99.145820,ALAMEDA,9m,27,0,1,0,0,...,1,0,0,4,0,0,1,0,0,0
2,3,19.434573,-99.143858,ALAMEDA,9m,15,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
3,4,19.434374,-99.142704,ALAMEDA,9m,16,0,1,0,0,...,2,0,1,2,0,0,1,0,0,0
4,5,19.434242,-99.141880,ALAMEDA,9m,13,0,1,0,0,...,2,0,2,3,0,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,12318,19.435075,-99.119716,CONGRESO,9m,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
442,12578,19.439129,-99.139482,CENTRO,9m,0,0,0,0,0,...,1,0,1,1,0,0,0,0,0,0
443,13795,19.436069,-99.121176,CONGRESO,9m,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
444,16948,19.434749,-99.144808,ALAMEDA,9m,21,0,0,0,0,...,1,0,0,2,0,0,0,0,0,0


#### Agrupar los puntos de interes

In [4]:
IP_groups = [['C09-CUARTELES DE LA POLICÍA AUXILIAR','C10-CUARTELES PBI','C11-CUARTELES PGJ','C16-JUZGADOS CIVILES Y PENALES','C20-MINISTERIOR PUBLICOS','C21-MODULOS SSP',],	# P01 - Seguridad

['C05-CENTRALES CAMIONERAS','C19-METROBUS','C31-TREN LIGERO','C42-TROLEBUS','C43-TURIBUS','C72-ACCESOS METRO','C81-ESTACIONES DE CABLEBUS',],										# P02 - Transporte

['C06-CENTROS COMERCIALES','C38-MERCADOS PUBLICOS','C49-TIENDAS DEPARTAMENTALES','C53-CINES','C62-OXXO',],#'C46-ESTABLECIMIENTOS MERCANTILES',],									# P03 - Comercio

['C22-MONUMENTOS HISTORICOS', 'C57-EVENTOS MASIVOS', 'C73-ATRACTIVOS TURISTICOS', 'C78-ZONAS ARQUEOLOGICAS',],																		# P04 - Turismo

['C58-CASAS Y CENTROS DE CULTURA','C59-MUSEOS Y TEATROS','C75-FONOTECAS','C76-FOTOTECA','C77-GALERIAS', 'C26-PLAZAS Y PARQUES',],													# P05 - Cultura

['C23-NOTARIAS','C24-OFICINAS DE GOBIERNO','C27-RECLUSORIOS','C30-TESORERIAS','C82-ALCALDIAS', 'C01-BANCOS'],																		# P06 - Gobierno

['C15-HOSPITALES','C45-CENTROS DE SALUD Y CLINICAS',],																																# P07 - Hospitales

['C44-GUARDERIAS', 'C67-CENTROS PILARES', 'C13-EDUCACION',],#'C13-CAM', 'C13-CPARA EL TRABAJO', 'C13_PREESC', 'C13_PRIM', 'C13_SEC', 'C13_BACH', 'C13_SUP',],											# P08 - Escuelas

['C33-IGLESIAS Y TEMPLOS',],																																						# P09 - Iglesias

['C51-EDIFICIOS',],]																																	# P10 - Edificios

# Cada nueva columna es la suma de las columnas que se encuentran en la lista de agrupaciones con nombre GRUPO-k
for i in range(len(IP_groups)):
	data_interest_points['P.Interes-'+str(i+1)] = data_interest_points[IP_groups[i]].sum(axis=1)
	# # Eliminamos las columnas que ya no se van a utilizar
	data_interest_points = data_interest_points.drop(IP_groups[i], axis=1)
data_interest_points = data_interest_points.drop(["C46-ESTABLECIMIENTOS MERCANTILES"], axis=1)

# Estandarizamos los datos
scaler = StandardScaler(with_mean=True, with_std=True)
data_interest_points[data_interest_points.filter(regex='^P.Interes-\d+').columns] = scaler.fit_transform(data_interest_points[data_interest_points.filter(regex='^P.Interes-\d+').columns])
data_interest_points

,id_camara,latitud,longitud,sector,tipo,P.Interes-1,P.Interes-2,P.Interes-3,P.Interes-4,P.Interes-5,P.Interes-6,P.Interes-7,P.Interes-8,P.Interes-9,P.Interes-10
0,1,19.435283,-99.147152,ALAMEDA,9m,1.289329,0.704281,3.231637,3.903102,0.934393,2.399202,-0.491347,0.931607,-0.778049,-0.294358
1,2,19.435098,-99.145820,ALAMEDA,9m,1.289329,-0.042709,1.387350,3.274320,0.692945,2.646984,-0.491347,0.931607,-0.778049,-0.294358
2,3,19.434573,-99.143858,ALAMEDA,9m,1.289329,-0.416204,0.926278,2.959929,0.210049,1.284186,-0.491347,-0.397983,0.644123,-0.294358
3,4,19.434374,-99.142704,ALAMEDA,9m,-0.335080,-0.042709,2.309493,2.016756,0.210049,1.408077,-0.491347,-0.397983,0.644123,-0.294358
4,5,19.434242,-99.141880,ALAMEDA,9m,-0.335080,0.704281,2.770565,2.016756,0.934393,1.036404,-0.491347,-0.397983,2.066295,-0.294358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,12318,19.435075,-99.119716,CONGRESO,9m,-0.335080,0.330786,-0.456936,-0.498373,-0.755743,-0.698066,-0.491347,-1.062777,-0.778049,-0.294358
442,12578,19.439129,-99.139482,CENTRO,9m,-0.335080,1.451271,-0.456936,-0.183982,0.210049,-0.698066,1.722196,0.931607,0.644123,-0.294358
443,13795,19.436069,-99.121176,CONGRESO,9m,-0.335080,-0.042709,-0.918008,-0.498373,-0.755743,-0.698066,-0.491347,0.931607,-0.778049,-0.294358
444,16948,19.434749,-99.144808,ALAMEDA,9m,1.289329,-0.416204,0.465207,3.588711,0.210049,2.027530,-0.491347,-0.397983,0.644123,-0.294358


In [5]:
data['date'] = data['fecha_creacion'].dt.strftime('%Y-%m-%d').astype('datetime64[ns]')
camaras_fechas_frecuencias = data.groupby(['id_camara', 'date']).size().reset_index(name='n_incidentes')

# Convertir la columna de fecha a datetime si aún no lo es
camaras_fechas_frecuencias['date'] = pd.to_datetime(camaras_fechas_frecuencias['date'])

# Crear un DataFrame con todas las fechas posibles

fechas = pd.date_range(start='2021-01-01', end=camaras_fechas_frecuencias['date'].max().date().isoformat())

# Obtener los id únicos de camara
camaras = camaras_fechas_frecuencias['id_camara'].unique()

# Crear un DataFrame con todas las combinaciones posibles de id_camara y fechas
df_total = pd.DataFrame(index=pd.MultiIndex.from_product([camaras, fechas], names=['id_camara', 'date'])).reset_index()

# Unir el DataFrame total con el DataFrame original
data_final = pd.merge(df_total, camaras_fechas_frecuencias, how='left', on=['id_camara', 'date'])

# Llenar los valores nulos con 0
data_final['n_incidentes'] = data_final['n_incidentes'].replace(np.nan, 0)

# Ahora desagregamos la fecha
data_final['anio'] = data_final['date'].dt.year
data_final['mes'] = data_final['date'].dt.month
data_final['dia_semana'] = data_final['date'].dt.dayofweek+1
data_final['dia'] = data_final['date'].dt.day
# data_final['hora'] = data_final['date'].dt.hour

# data_final.drop(['date'], axis=1, inplace=True)



data_final

,id_camara,date,n_incidentes,anio,mes,dia_semana,dia
0,1,2021-01-01,0.0,2021,1,5,1
1,1,2021-01-02,0.0,2021,1,6,2
2,1,2021-01-03,0.0,2021,1,7,3
3,1,2021-01-04,0.0,2021,1,1,4
4,1,2021-01-05,0.0,2021,1,2,5
...,...,...,...,...,...,...,...
365269,17290,2023-03-26,0.0,2023,3,7,26
365270,17290,2023-03-27,0.0,2023,3,1,27
365271,17290,2023-03-28,0.0,2023,3,2,28
365272,17290,2023-03-29,0.0,2023,3,3,29


#### Asignar los puntos de interes a la cada registro de la base de datos de 'data'

In [6]:
# Recorrer los registros del df data y agregar las columnas de P.Interes de acuerdo a la columna id_camara
data_final[data_interest_points.filter(regex='^P.Interes-\d+').columns] = 0
for i in range(len(data_final)):
	id_camara = data_final['id_camara'][i]
	# Buscar el id_camara en el df data_interest_points
	index = data_interest_points[data_interest_points['id_camara'] == id_camara].index
	if len(index) > 0:
		for column in data_interest_points.filter(regex='^P.Interes-\d+').columns:
			data_final[column][i] = data_interest_points[column][index[0]]

data_final['n_incidentes'] = data_final['n_incidentes'].apply(lambda x: 0 if x == 0 else 1)

data = data_final.copy()
data

C:\Users\jhernandeza\AppData\Local\Temp\ipykernel_5384\2438913492.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_final[column][i] = data_interest_points[column][index[0]]


,id_camara,date,n_incidentes,anio,mes,dia_semana,dia,P.Interes-1,P.Interes-2,P.Interes-3,P.Interes-4,P.Interes-5,P.Interes-6,P.Interes-7,P.Interes-8,P.Interes-9,P.Interes-10
0,1,2021-01-01,0.0,2021,1,5,1,1.289329,0.704281,3.231637,3.903102,0.934393,2.399202,-0.491347,0.931607,-0.778049,-0.294358
1,1,2021-01-02,0.0,2021,1,6,2,1.289329,0.704281,3.231637,3.903102,0.934393,2.399202,-0.491347,0.931607,-0.778049,-0.294358
2,1,2021-01-03,0.0,2021,1,7,3,1.289329,0.704281,3.231637,3.903102,0.934393,2.399202,-0.491347,0.931607,-0.778049,-0.294358
3,1,2021-01-04,0.0,2021,1,1,4,1.289329,0.704281,3.231637,3.903102,0.934393,2.399202,-0.491347,0.931607,-0.778049,-0.294358
4,1,2021-01-05,0.0,2021,1,2,5,1.289329,0.704281,3.231637,3.903102,0.934393,2.399202,-0.491347,0.931607,-0.778049,-0.294358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365269,17290,2023-03-26,0.0,2023,3,7,26,-0.335080,-0.416204,-0.918008,-0.498373,-0.514295,-0.698066,-0.491347,-1.062777,-0.778049,-0.294358
365270,17290,2023-03-27,0.0,2023,3,1,27,-0.335080,-0.416204,-0.918008,-0.498373,-0.514295,-0.698066,-0.491347,-1.062777,-0.778049,-0.294358
365271,17290,2023-03-28,0.0,2023,3,2,28,-0.335080,-0.416204,-0.918008,-0.498373,-0.514295,-0.698066,-0.491347,-1.062777,-0.778049,-0.294358
365272,17290,2023-03-29,0.0,2023,3,3,29,-0.335080,-0.416204,-0.918008,-0.498373,-0.514295,-0.698066,-0.491347,-1.062777,-0.778049,-0.294358


In [16]:


#Split data into train and test, train will be until 2022 and test will be rest of the data
train = data[data['date'] < '2023-01-01']
test = data[data['date'] >= '2023-01-01']

#Split train and test into X and y
X_train = train.drop(['id_camara', 'n_incidentes', 'date'], axis=1)
y_train = train['n_incidentes']
X_test = test.drop(['id_camara', 'n_incidentes', 'date'], axis=1)
y_test = test['n_incidentes']
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


(325580, 14) (325580,) (39694, 14) (39694,)


### Oversampling

In [25]:
# Oversampling para las clases minoritarias
import imblearn
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

# Resumen de la distribución de clases
from collections import Counter
counter = Counter(y_train)
print(counter)


Counter({0: 301162, 1: 301162})


### Modelo de Red Neuronal - Regresion Logistica

In [26]:
from keras import backend as K

def f1_score(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

# Crear red neuronal de regresión logística
# Definir el modelo
model = Sequential()
model.add(Dense(1, input_shape=(X_train.shape[1],), activation='relu'))

# Añadir la capa de entrada y una capa oculta
model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))

# Añadir otra capa oculta
model.add(Dense(16, activation='relu'))
# Añadir la capa de salida binaria, 0 o 1, no numeros decimales
model.add(Dense(1, activation='sigmoid'))



# Para condicion de paro en convergencia realizar el siguiente paso
# En la linea 

# Compilar el modelo
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy', AUC(name='auc'), Precision(name='precision'), Recall(name='recall'), f1_score])

# Crear el callback para guardar el modelo
checkpoint = ModelCheckpoint('model-{epoch:03d}.h5', verbose=0, monitor='uac', save_best_only=True, mode='auto')
# Entrenar el modelo
# model.fit(data.drop(['id_camara','date','n_incidentes'], axis=1), data['n_incidentes'], epochs=100, callbacks=[checkpoint], validation_split=0.2)
model.fit(X_train, y_train, epochs=100, callbacks=[checkpoint], validation_split=0.2)

# Guardar el modelo
model.save('model.h5')

# Cargar el modelo (Recuerda importar la librería: from keras.models import load_model)
model = load_model('model.h5', custom_objects={'f1_score': f1_score})

# evaluar el modelo
# model.evaluate(data.drop(['id_camara'], axis=1), data['id_camara'])
model.evaluate(X_train, y_train) # 

Epoch 1/100
15059/15059 [==============================] - 34s 2ms/step - loss: 0.6621 - accuracy: 0.6250 - auc: 0.4997 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.0000e+00 - val_loss: 0.9756 - val_accuracy: 0.0000e+00 - val_auc: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.0000e+00
Epoch 2/100
15059/15059 [==============================] - 30s 2ms/step - loss: 0.6616 - accuracy: 0.6250 - auc: 0.4998 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.0000e+00 - val_loss: 0.9829 - val_accuracy: 0.0000e+00 - val_auc: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.0000e+00
Epoch 3/100
15059/15059 [==============================] - 30s 2ms/step - loss: 0.6616 - accuracy: 0.6250 - auc: 0.4994 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.0000e+00 - val_loss: 0.9827 - val_accuracy: 0.0000e+00 - val_auc: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.0

[0.7259893417358398, 0.5, 0.5, 0.0, 0.0, 0.0]

In [37]:
# Predecir
probs = model.predict(X_test)
probs
labels = (probs > 0.5).astype(int)
# predictions = predictions.reshape((predictions.shape[0],))
# np.unique(predictions, return_counts=True)
# predictions = np.round(predictions)
# predictions = predictions.astype(int)
# np.unique(labels, return_counts=True)

1241/1241 [==============================] - 1s 1ms/step


In [36]:
# Calcular el accuracy
print("Accuracy:",metrics.accuracy_score(y_test, labels))

# Calcular la matriz de confusión
print(metrics.confusion_matrix(y_test, labels, labels=y_test.unique()))

Accuracy: 0.938882450748224
[[37268     0]
 [ 2426     0]]
